In [1]:
import spconv.pytorch as spconv
import torch
from torch import nn

In [2]:
torch.manual_seed(2024)

In [3]:
spconv3d = spconv.SparseConv3d(
    in_channels=5, 
    out_channels=16, 
    kernel_size=3, 
    stride=1, 
    padding=0,
    bias=False
)

In [4]:
spconv3d.weight.shape

torch.Size([16, 3, 3, 3, 5])

In [5]:
conv3d = nn.Conv3d(
    in_channels=5, 
    out_channels=16, 
    kernel_size=3, 
    stride=1, 
    padding=0,
    bias=False
)

In [6]:
sp_weight = spconv3d.weight.data.clone()

In [7]:
sp_weight.shape

torch.Size([16, 3, 3, 3, 5])

In [8]:
weight = sp_weight.permute(0, 4, 1, 2, 3)

In [9]:
conv3d.weight.data = weight

In [10]:
conv3d.to('cuda')
spconv3d.to('cuda')

SparseConv3d(5, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[0, 0, 0], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)

In [11]:
# Parameters
N = 10  # number of non-zero points
num_channels = 5  # number of feature channels per point
ndim = 3  # number of spatial dimensions (e.g., 3D convolution)
batch_size = 2  # number of batches

In [12]:
# Generating random features
features = torch.randn(N, num_channels).to('cuda')

In [13]:
features.shape

torch.Size([10, 5])

In [14]:
# Generating random indices for a 3D sparse tensor with batch indices
spatial_shape = (100, 100, 100)  # Define the spatial shape of the sparse tensor
indices = torch.zeros(N, ndim + 1, dtype=torch.int32).to('cuda')
indices[:, 0] = torch.randint(0, batch_size, (N,))  # Batch indices
for i in range(1, ndim + 1):
    indices[:, i] = torch.randint(0, spatial_shape[i - 1], (N,))  # Spatial indices

In [15]:
# Creating the SparseConvTensor
x = spconv.SparseConvTensor(features, indices, spatial_shape, batch_size)

In [16]:
x

SparseConvTensor[shape=torch.Size([10, 5])]

In [17]:
# Converting sparse tensor to dense NCHW tensor (just for visualization; might be memory intensive)
x_dense_NCHW = x.dense()
print(x_dense_NCHW.shape)  # Should match the expected dense shape [batch_size, num_channels, *spatial_shape]

torch.Size([2, 5, 100, 100, 100])


In [18]:
x_dense_NCHW = x_dense_NCHW.permute(0, 2, 3, 4, 1)

In [19]:
sparse_tensor = spconv.SparseConvTensor.from_dense(x_dense_NCHW)

In [20]:
y = spconv3d(x)
sparse_y = spconv3d(sparse_tensor)

In [21]:
y_dense = y.dense()
sparse_y_dense = sparse_y.dense()

In [22]:
torch.allclose(y_dense, sparse_y_dense)

True

In [23]:
x_dense_NCHW = x_dense_NCHW.permute(0, 4, 1, 2, 3)

In [24]:
conv_y = conv3d(x_dense_NCHW)

In [25]:
torch.allclose(conv_y, sparse_y_dense)

False

In [26]:
conv_y.unique()

tensor([-0.3298, -0.3051, -0.3035,  ...,  0.3083,  0.3124,  0.3248],
       device='cuda:0', grad_fn=<Unique2Backward0>)

In [27]:
sparse_y_dense.unique()

tensor([-0.3298, -0.3050, -0.3035,  ...,  0.3083,  0.3124,  0.3248],
       device='cuda:0', grad_fn=<Unique2Backward0>)

In [28]:
l1loss = torch.nn.L1Loss()

In [29]:
l1loss(conv_y, sparse_y_dense)

tensor(2.8299e-09, device='cuda:0', grad_fn=<MeanBackward0>)